<a href="https://colab.research.google.com/github/bdgToji/VNP/blob/main/KOL2_Zadaca2_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Симнете го податочното множество

In [1]:
!gdown 1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF

Downloading...
From: https://drive.google.com/uc?id=1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF
To: /content/IMDB_5.csv
100% 323k/323k [00:00<00:00, 102MB/s]


# Направете претпроцесирање на податоците

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/content/IMDB_5.csv")

In [4]:
df

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",negative
1,Why do movie makers always go against the auth...,negative
2,No puedo creer que aquellos alabando esta pelí...,negative
3,Esta película realmente usó sus ubicaciones bi...,positive
4,"De manera extraña, esta película nunca llegó a...",positive
...,...,...
245,Obabakoak es un montón de cuentos con un único...,negative
246,Todo el mundo civilizado por ahora sabe que es...,positive
247,"Soy un enorme fan de las películas de guerra, ...",negative
248,"De acuerdo, aquí está el trato: si te ofenden ...",positive


In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
encoder = LabelEncoder()

df['sentiment'] = encoder.fit_transform(df['sentiment'])

In [7]:
df.head()

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",0
1,Why do movie makers always go against the auth...,0
2,No puedo creer que aquellos alabando esta pelí...,0
3,Esta película realmente usó sus ubicaciones bi...,1
4,"De manera extraña, esta película nunca llegó a...",1


In [8]:
df.columns = ['review_es', 'label']

In [9]:
df.head()

,review_es,label
0,"¡Esta película es horrible, ni siquiera puedo ...",0
1,Why do movie makers always go against the auth...,0
2,No puedo creer que aquellos alabando esta pelí...,0
3,Esta película realmente usó sus ubicaciones bi...,1
4,"De manera extraña, esta película nunca llegó a...",1


# Поделето го на тренирачко и тестирачко множество во сооднос 80:20

In [10]:
!pip install transformers datasets
!pip install transformers[torch]
!pip install accelerate -U
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [13]:
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

dataset = load_dataset("csv", data_files={"train":"train.csv", "test":"test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_es', 'label'],
        num_rows: 200
    })
    test: Dataset({
        features: ['review_es', 'label'],
        num_rows: 50
    })
})

# Искористете соодветен трансформер модел за проблемот

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [16]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
encoded_dataset = dataset.map(lambda t: tokenizer(t["review_es"], truncation=True), batched=True, load_from_cache_file=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [17]:
model = AutoModelForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Фино-подесете (тренирајте) го моделот

In [18]:
from transformers import TrainingArguments, Trainer

In [19]:
args = TrainingArguments(
    "sentiment",
    num_train_epochs=5,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    seed=42
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = encoded_dataset['train'],
    eval_dataset = encoded_dataset['test'],
    tokenizer = tokenizer
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=125, training_loss=0.2883105163574219, metrics={'train_runtime': 84.7695, 'train_samples_per_second': 11.797, 'train_steps_per_second': 1.475, 'total_flos': 243163948475520.0, 'train_loss': 0.2883105163574219, 'epoch': 5.0})

# Тестирајте го моделот на тестирачкото множество

In [20]:
predictions = trainer.predict(encoded_dataset['test'])
preds = np.argmax(predictions.predictions, axis=-1)

# Пресметајте f1 score

In [21]:
from sklearn.metrics import classification_report, f1_score

In [23]:
print(classification_report(predictions.label_ids, preds))

              precision    recall  f1-score   support

           0       0.96      0.81      0.88        27
           1       0.81      0.96      0.88        23

    accuracy                           0.88        50
   macro avg       0.89      0.89      0.88        50
weighted avg       0.89      0.88      0.88        50



In [24]:
print(f1_score(predictions.label_ids, preds))

0.8800000000000001
